#### 同策略基金分析

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from WindPy import w
w.start()
from function import *
from nav_research import NavResearch
from pathlib import Path
import os
import shutil
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.globals import ThemeType
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [2]:
basic_info = load_data(r"data\产品目录.xlsx")
basic_info

,strategy,fund_code,fund_name,benchmark_code,benchmark_name,threshold
0,主观价值,SQJ763,睿郡财富11号1期,HSI.HI,恒生指数,-0.10
1,主观价值,SS5163,红筹长线3号,000300.SH,沪深300,-0.10
2,量化CTA,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.10
3,套利,SZH690,辰元优选稳健,H11009.CSI,中证综合债,-0.03
4,主观CTA,SM6151,草本优益1号,NH0100.NHF,南华商品指数,-0.10
5,主观CTA,SM6151,草本致远1号,NH0300.NHF,南华农产品指数,-0.10


In [3]:
strategy_info = basic_info.loc[(basic_info["strategy"] == "主观价值"), :]
strategy_info

,strategy,fund_code,fund_name,benchmark_code,benchmark_name,threshold
0,主观价值,SQJ763,睿郡财富11号1期,HSI.HI,恒生指数,-0.1
1,主观价值,SS5163,红筹长线3号,000300.SH,沪深300,-0.1


In [4]:
data = None
for fund in strategy_info["fund_name"]:
    nav_data_path = os.path.join(r"C:\Users\17820\Desktop\VScode\Private_nav_research\data",fund +".xlsx")
    df = load_data(nav_data_path)
    df = get_standardized_data(df)
    df.rename(columns={"date":"日期", "nav_adjusted":fund}, inplace=True)
    df = df[["日期",fund]]
    if data is None:
        data = df
    else:
        data = pd.merge(data, df, on="日期", how="inner")
        data = data.set_index("日期")
        df_normalized = data.div(data.iloc[0])

In [5]:
get_line(df_normalized, "主观CTA").render_notebook()